In [251]:
# Imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.stats import distributions as pdf
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import sklearn.datasets as dset

In [274]:
# helper functions
def make_data(num_samples, scatter):
    np.random.seed(123)
    class_means =[(1,8),(1,1),(8,1),(30,8)]
    c1_samples =np.random.multivariate_normal(mean=class_means[0], 
                                             cov=[[scatter[0] *1, 0], [0, scatter[0] *1]], 
                                            size=num_samples[0])
    
    c2_samples = np.random.multivariate_normal(mean=class_means[1], 
                                            cov=[[scatter[1] *1, 0], [0, scatter[1] *1]], 
                                            size=num_samples[1])
    
    c3_samples = np.random.multivariate_normal(mean=class_means[2], 
                                            cov=[[scatter[2] *1, 0], [0, scatter[2] *1]], 
                                            size=num_samples[2])
    
    c4_samples = np.random.multivariate_normal(mean=class_means[3], 
                                            cov=[[scatter[3] *1, 0], [0, scatter[3] *1]], 
                                            size=num_samples[3])
    
    # labels
    c1_labels = np.vstack((np.ones(c1_samples.shape[0]), 
                           np.zeros(c1_samples.shape[0]), 
                           np.zeros(c1_samples.shape[0]),
                           np.zeros(c1_samples.shape[0]))).T
    c2_labels = np.vstack((np.zeros(c2_samples.shape[0]), 
                           np.ones(c2_samples.shape[0]), 
                           np.zeros(c2_samples.shape[0]),
                           np.zeros(c2_samples.shape[0]))).T
    c3_labels = np.vstack((np.zeros(c3_samples.shape[0]), 
                           np.zeros(c3_samples.shape[0]), 
                           np.ones(c3_samples.shape[0]),
                           np.zeros(c3_samples.shape[0]))).T
    c4_labels = np.vstack((np.zeros(c4_samples.shape[0]), 
                           np.zeros(c4_samples.shape[0]), 
                           np.zeros(c4_samples.shape[0]),
                           np.ones(c4_samples.shape[0]))).T
    
    
    # feature matrix, weight matrix, label matrix (one-hot)
    x = np.vstack((c1_samples, c2_samples, c3_samples, c4_samples))
    x = np.hstack((np.ones(x.shape[0])[:,None],x)) # bias term
    labels = np.vstack((c1_labels, c2_labels, c3_labels, c4_labels))
    return x, labels, (c1_samples, c2_samples, c3_samples, c4_samples)

def plot_data(c1_samples,c2_samples,c3_samples, c4_samples):
    ax = plt.subplot()
#     c1_samples,c2_samples,c3_samples=np.log(c1_samples),np.log(c2_samples),np.log(c3_samples)

    ax.scatter(c1_samples[:,0],c1_samples[:,1])
    ax.scatter(c2_samples[:,0],c2_samples[:,1])
    ax.scatter(c3_samples[:,0],c3_samples[:,1])
    ax.scatter(c4_samples[:,0],c4_samples[:,1])
    return ax


def loss_fn(x,weights,labels):    
    # scores
    s = x@weights.T  # scores

    eps = np.max(s)  # constant for numerical stability
    probs = (np.exp(s-eps))/ (np.sum(np.exp(s-eps), axis=1))[:,None]
    # Loss function (negative log-likelihood)
    loss = np.sum(-labels*np.log(probs))
    dloss_dw = (labels*(probs-labels)).T@x
    return loss, dloss_dw, probs


def probability(x,weights):
    s = x@weights.T  # scores
    eps = np.max(s)  # constant for numerical stability
    probs = (np.exp(s-eps))/ (np.sum(np.exp(s-eps), axis=1))[:,None]
    return probs


def grid(x,w, mode='sep'):
    
    h = .02  # step size in the mesh
    x_min, x_max = x[:, 1].min() - 1, x[:, 1].max() + 1
    y_min, y_max = x[:, 2].min() - 1, x[:, 2].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    x_tmp=np.c_[xx.ravel(), yy.ravel()]
    x_tmp = np.hstack((np.ones(x_tmp.shape[0])[:,None],x_tmp)) # bias term

    pr = probability(x_tmp,w)

    z = np.argmax(pr, axis=1)
    
    z = z.reshape(xx.shape)

#     for i, ig in enumerate(grid_x):
#         for j, jg in enumerate(grid_y):
#             feats = np.array((1,ig,jg))[:,None].T
#             pr=probability(feats,w)
#             if mode == 'sep':
#                 grid[i,j]=np.argmax(pr)
#             elif mode == 'probs':
#                 grid[i,j]=np.max(pr)

    return xx, yy, z
  

def plot_decision_boundary(ax1, u, v, z, levels=None, colors=['k','k','k']):
    # Basic contour plot
#     fig1, ax1 = plt.subplots()
    if levels == None:
        ctp = ax1.contour(u, v, z, alpha=0.6)
    else:
        ctp = ax1.contour(u, v, z, levels=levels, colors=colors, alpha=0.6)
        # Description of contours
        fmt = {l: str(l) for l in ctp.levels}
        ax1.clabel(ctp, ctp.levels, inline=True, fmt=fmt, fontsize=10)
    return ax1

In [275]:
@interact(num_epochs=(0,10000,500), 
          learning_rate=[0.01,0.02,0.03], 
          c1_samples='20',
          c1_scatter=(1,8,0.4),
          c2_samples='20',
          c2_scatter=(1,8,0.4),
          c3_samples='20',
          c3_scatter=(1,20,0.4),
          c4_samples='20',
          c4_scatter=(1,8,0.4),)
def multi_log_regression(num_epochs, 
                         learning_rate, 
                         c1_samples,
                         c1_scatter,
                         c2_samples, 
                         c2_scatter,
                         c3_samples,
                         c3_scatter,
                         c4_samples,
                         c4_scatter):
    if  c1_samples.isdigit()==False or c2_samples.isdigit()==False or c3_samples.isdigit()==False or c4_samples.isdigit()==False:
            pass
    else:
        c1_samples = int(c1_samples)
        c2_samples = int(c2_samples)
        c3_samples = int(c3_samples)
        c4_samples = int(c4_samples)
        x, labels, samples = make_data((c1_samples, c2_samples, c3_samples, c4_samples),
                                       (c1_scatter, c2_scatter, c3_scatter, c4_scatter))

        w = 0.001 * np.ones((4,x.shape[1])) # #classes x features
        for epoch in range(num_epochs):
                loss, dw, probs = loss_fn(x,w,labels)
                acc = np.sum(np.argmax(labels)==np.argmax(probs))
#                 print(w)
                w -= learning_rate*dw
        u,v,z = grid(x,w,mode='sep')
        acc = np.sum(np.argmax(labels, axis=1)==np.argmax(probs, axis=1))
        acc /= x.shape[0]
        

        ax = plot_data(*samples)
        
        # Plot missclassified data
        if acc != 1.0:
            idx = np.argmax(labels, axis=1)!=np.argmax(probs, axis=1)
            ax.scatter(x[idx,1], x[idx,2], marker='x', c='k')
        
        try: 
            ax.text(9,10,'loss: {0:.4f}'.format(loss))
            ax.text(9,11,'acc: {0:.4f}'.format(acc)) 
        except:
            pass

        ax = plot_decision_boundary(ax, u,v,z)

        
        plt.xlim(-8,30)
        plt.ylim(-8,15)


        ax.figure
        
#     return ax
    

interactive(children=(IntSlider(value=5000, description='num_epochs', max=10000, step=500), Dropdown(descripti…

In [261]:
w_tmp = np.array([[  8.64093347,  -0.12633717,  -1.04936902],
 [  2.98071988 , -1.6248274  ,  1.19934607],
 [-11.61865335 ,  1.75416458 , -0.14697705]])

In [255]:
x=np.array((1,3.73346091,14.25079417))[:,None]

In [151]:
probability(x.T, w_tmp)

array([[3.26863738e-02, 9.67312898e-01, 7.28159829e-07]])

In [150]:
x=np.array((1,2.38405744,5.61221124))[:,None]

In [206]:
ts = np.array([[  8.64093347,  -0.12633717,  -1.04936902],
 [  2.98071988 , -1.6248274  ,  1.19934607],
 [-11.61865335 ,  1.75416458 , -0.14697705]])
fd = np.array([[  0,  0,  1],
 [  0 , 1  ,  0],
 [1,  0 , 0]])
print(ts[fd==1].shape)


(3,)
